#Comparativa clasificación de textos

En esta parte, utilizaremos **embeddings** para resolver un problema de clasificación de texto. Los embeddings, representaciones distribuidas y vectoriales de elementos, son un concepto muy común en el mundo del deep learning. Los **word vectors** que hemos visto en clase son una representación en forma de embedding de las palabras.


Vamos a utilizar el dataset **"Reuters newswire topics classification"**, disponible desde Keras de manera similar al dataset de IMDB ([ver documentación](https://keras.io/datasets/#reuters-newswire-topics-classification)).

---

Tenemos varias opciones para entrenar modelos con embeddings. 

*   Utilizar una **Media de Embeddings** .
*  Utilizar una **RNN** sobre una secuencia de word vectors. Un buen consejo es emplear una red recurrente bidireccional.
*   Utilizar una **CNN** sobre una secuencia de word vectors. Aquí necesitamos cambiar un poco la idea de convolución para actuar sobre sequencias de vectores. Keras incluye una [Convolución en 1D](https://keras.io/layers/convolutional/#conv1d) que puede ser utilizada en este caso, con un ejemplo de uso en la documentación. Una forma de hacer funcionar este esquema sería utilizar la convolución en 1D + max pooling.
*  En esta práctica se pide implementar estos 3 modelos. Teniendo que ser el Acurracy de los datos Test por encima del **67 %**, en caso de no llegar en el primer modelo se deberá de realizar experimentos hasta llegar al desempeño objetivo.
---

Dos **hiperparámetros** importantes a elegir en el modelo son la **longitud de las secuencias de texto** y el **tamaño del vocabulario** para los embeddings. Nótese que, al cortar todas las secuencias para que tengan el mismo tamaño, podríamos estar perdiendo mucho texto si elegimos un tamaño de secuencia demasiado pequeño. Igualmente, si las hacemos muy largas necesitaremos más tiempo para entrenar nuestros modelos. Una buena idea consiste en explorar los datos para ver cómo suelen ser de largos los textos y encontrar un buen trade-off para el tamaño de al secuencia.

No utilizar Early Stopping y utilizar 50 epocas para los 3 tipos de modelos.

---

Los embeddings  se entrenan junto al modelo.  Una técnica frecuente es inicializar estos embeddings con word-vectors pre-entrenados en un gran corpus de texto. Esto puede ayudar ya que nuestro modelo empieza con unos embeddings que ya encapsulan significado. Si bien no es necesario para esta práctica, podéis ver cómo usar esta técnica [en el siguiente tutorial](https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html).




# Experimento 1: Media de Embeddings.

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from keras.callbacks import EarlyStopping
from prettytable import PrettyTable


reuters = keras.datasets.reuters

In [ ]:
from numpy.matrixlib.defmatrix import matrix
#Con cuentas palabras voy a trabahar
VOCAB_SIZE =1000
SEQUENCE_LENGHT = 150
(train_data, train_labels),(test_data, test_labels) = reuters.load_data(num_words=VOCAB_SIZE)
print("Training entries: {}, Test entries: {}".format(len(train_data),len(test_data)))

#Tamaños de los textos para determinar longitud de las secuencias a usar

lenghts = [len(x) for x in train_data]
print("Average length es {}".format(np.mean(lenghts)))
print("Max length es {}".format(max(lenghts)))
print("Median length es {}".format(np.percentile(lenghts, 50)))


Training entries: 8982, Test entries: 2246
Average length es 145.5398574927633
Max length es 2376
Median length es 95.0


In [ ]:
#Nos hace la conversión. Cada índice esta asociado a un word vector, índice a partir del cual se leen los word embedding asociados
train_data

array([list([1, 2, 2, 8, 43, 10, 447, 5, 25, 207, 270, 5, 2, 111, 16, 369, 186, 90, 67, 7, 89, 5, 19, 102, 6, 19, 124, 15, 90, 67, 84, 22, 482, 26, 7, 48, 4, 49, 8, 864, 39, 209, 154, 6, 151, 6, 83, 11, 15, 22, 155, 11, 15, 7, 48, 9, 2, 2, 504, 6, 258, 6, 272, 11, 15, 22, 134, 44, 11, 15, 16, 8, 197, 2, 90, 67, 52, 29, 209, 30, 32, 132, 6, 109, 15, 17, 12]),
       list([1, 2, 699, 2, 2, 56, 2, 2, 9, 56, 2, 2, 81, 5, 2, 57, 366, 737, 132, 20, 2, 7, 2, 49, 2, 2, 2, 2, 699, 2, 8, 7, 10, 241, 16, 855, 129, 231, 783, 5, 4, 587, 2, 2, 2, 775, 7, 48, 34, 191, 44, 35, 2, 505, 17, 12]),
       list([1, 53, 12, 284, 15, 14, 272, 26, 53, 959, 32, 818, 15, 14, 272, 26, 39, 684, 70, 11, 14, 12, 2, 18, 180, 183, 187, 70, 11, 14, 102, 32, 11, 29, 53, 44, 704, 15, 14, 19, 758, 15, 53, 959, 47, 2, 15, 14, 19, 132, 15, 39, 965, 32, 11, 14, 147, 72, 11, 180, 183, 187, 44, 11, 14, 102, 19, 11, 123, 186, 90, 67, 960, 4, 78, 13, 68, 467, 511, 110, 59, 89, 90, 67, 2, 55, 2, 92, 617, 80, 2, 46, 905, 220, 13,

In [ ]:
test_labels


array([ 3, 10,  1, ...,  3,  3, 24])

In [ ]:
word_index = reuters.get_word_index()

#Los primeros índices son reservados. 
#Asignamos un valor para inicio de cadena, padding. A nuestro word index original le estamos añadiendo estos tokens especiales

word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0 
word_index["<START>"] = 1 
word_index["<UNK>"] = 2 #No conocido
word_index["<UNUSED>"] = 3

#Si sabemos un índice podemos saber la palabra. 

reverse_word_index = dict([(value,key) for (key,value) in word_index.items()])

def decode_review(text):
    return ''.join([reverse_word_index.get(i, '?') for i in text])


X_train = keras.preprocessing. sequence.pad_sequences(train_data, 
                                                        value=word_index["<PAD>"], 
                                                        padding='post', 
                                                        maxlen=SEQUENCE_LENGHT)

X_test = keras.preprocessing. sequence.pad_sequences(test_data, 
                                                        value=word_index["<PAD>"], 
                                                        padding='post', 
                                                        maxlen=SEQUENCE_LENGHT)

y_train= keras.utils.to_categorical(train_labels)
y_test= keras.utils.to_categorical(test_labels)

num_clases= y_train.shape[1]

print("Tenemos las siguientes clases {}".format(num_clases))


Tenemos las siguientes clases 46


In [ ]:
vocab_size = 10000

model = keras.Sequential()
 

model.add(keras.layers.Embedding(vocab_size,64)) 
#Los promedia los va a aplanar.
model.add(keras.layers.GlobalAveragePooling1D())
#Ahora si, una capa densa con relu
model.add(keras.layers.Dense(128, activation=tf.nn.relu)) # Nuestra ya conocida capa densa

model.add(keras.layers.Dense(num_clases, activation=tf.nn.softmax))

model.summary()

Model: "sequential_29"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_29 (Embedding)    (None, None, 64)          640000    
                                                                 
 global_average_pooling1d_15  (None, 64)               0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_52 (Dense)            (None, 128)               8320      
                                                                 
 dense_53 (Dense)            (None, 46)                5934      
                                                                 
Total params: 654,254
Trainable params: 654,254
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam', 
             loss='binary_crossentropy', 
             metrics=['accuracy'])


history = model.fit(X_train,
                    y_train,
                    epochs=50, 
                    batch_size=512, 
                    verbose=1,
                    validation_split=0.1)

Epoch 1/50
16/16 [==============================] - 3s 65ms/step - loss: 0.1033 - accuracy: 0.3177 - val_loss: 0.1013 - val_accuracy: 0.3315
Epoch 2/50
16/16 [==============================] - 0s 15ms/step - loss: 0.0980 - accuracy: 0.3540 - val_loss: 0.0929 - val_accuracy: 0.3315
Epoch 3/50
16/16 [==============================] - 0s 8ms/step - loss: 0.0853 - accuracy: 0.3540 - val_loss: 0.0764 - val_accuracy: 0.3315
Epoch 4/50
16/16 [==============================] - 0s 14ms/step - loss: 0.0713 - accuracy: 0.3540 - val_loss: 0.0710 - val_accuracy: 0.3315
Epoch 5/50
16/16 [==============================] - 0s 15ms/step - loss: 0.0686 - accuracy: 0.3540 - val_loss: 0.0690 - val_accuracy: 0.3315
Epoch 6/50
16/16 [==============================] - 0s 8ms/step - loss: 0.0671 - accuracy: 0.3540 - val_loss: 0.0680 - val_accuracy: 0.3315
Epoch 7/50
16/16 [==============================] - 0s 7ms/step - loss: 0.0658 - accuracy: 0.3639 - val_loss: 0.0668 - val_accuracy: 0.3737
Epoch 8/50
16/16

In [ ]:
results = model.evaluate(X_test, y_test)
print(results)
acurracyExp0=results[1]
myTable = PrettyTable(["Experimento", " Test Accuracy"])
myTable.add_row(["Media Embedings", "{0:.5f}".format(acurracyExp0)])
print(myTable)

71/71 [==============================] - 0s 2ms/step - loss: 0.0373 - accuracy: 0.7084
[0.03733041137456894, 0.7083704471588135]
+-----------------+----------------+
|   Experimento   |  Test Accuracy |
+-----------------+----------------+
| Media Embedings |    0.70837     |
+-----------------+----------------+


# Experimento 2: LSTM.

In [ ]:
model = keras.Sequential()
 

model.add(keras.layers.Embedding(vocab_size,64)) 
model.add(keras.layers.Bidirectional(keras.layers.LSTM(64)))
#Ahora si, una capa densa con relu
model.add(keras.layers.Dense(128, activation=tf.nn.relu)) # Nuestra ya conocida capa densa

model.add(keras.layers.Dense(num_clases, activation=tf.nn.softmax))

model.summary()

Model: "sequential_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_30 (Embedding)    (None, None, 64)          640000    
                                                                 
 bidirectional_6 (Bidirectio  (None, 128)              66048     
 nal)                                                            
                                                                 
 dense_54 (Dense)            (None, 128)               16512     
                                                                 
 dense_55 (Dense)            (None, 46)                5934      
                                                                 
Total params: 728,494
Trainable params: 728,494
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam', 
             loss='binary_crossentropy', 
             metrics=['accuracy'])


history = model.fit(X_train,
                    y_train,
                    epochs=50, 
                    batch_size=512, 
                    verbose=1,
                    validation_split=0.1)

Epoch 1/50
16/16 [==============================] - 6s 116ms/step - loss: 0.1006 - accuracy: 0.3095 - val_loss: 0.0845 - val_accuracy: 0.3315
Epoch 2/50
16/16 [==============================] - 1s 35ms/step - loss: 0.0747 - accuracy: 0.3267 - val_loss: 0.0707 - val_accuracy: 0.3537
Epoch 3/50
16/16 [==============================] - 1s 34ms/step - loss: 0.0653 - accuracy: 0.3896 - val_loss: 0.0609 - val_accuracy: 0.4883
Epoch 4/50
16/16 [==============================] - 0s 28ms/step - loss: 0.0584 - accuracy: 0.5062 - val_loss: 0.0584 - val_accuracy: 0.4917
Epoch 5/50
16/16 [==============================] - 0s 28ms/step - loss: 0.0549 - accuracy: 0.5249 - val_loss: 0.0531 - val_accuracy: 0.5562
Epoch 6/50
16/16 [==============================] - 0s 28ms/step - loss: 0.0505 - accuracy: 0.5644 - val_loss: 0.0510 - val_accuracy: 0.5662
Epoch 7/50
16/16 [==============================] - 0s 28ms/step - loss: 0.0486 - accuracy: 0.5763 - val_loss: 0.0499 - val_accuracy: 0.5873
Epoch 8/50
1

In [ ]:
results = model.evaluate(X_test, y_test)
print(results)
acurracyExp1=results[1]
myTable.add_row(["LSTM", "{0:.5f}".format(acurracyExp1)])
print(myTable)

71/71 [==============================] - 0s 6ms/step - loss: 0.0443 - accuracy: 0.6888
[0.04430641606450081, 0.6887800693511963]
+-----------------+----------------+
|   Experimento   |  Test Accuracy |
+-----------------+----------------+
| Media Embedings |    0.70837     |
|       LSTM      |    0.68878     |
+-----------------+----------------+


# Experimento 3: CNN.

In [ ]:
from keras.api._v2.keras import activations
vocab_size = 10000

model = keras.Sequential()
 

model.add(keras.layers.Embedding(vocab_size,64)) 
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Conv1D(filters=16,kernel_size=3,activation='relu'))
model.add(keras.layers.GlobalAveragePooling1D())

#Ahora si, una capa densa con relu
model.add(keras.layers.Dense(128, activation=tf.nn.relu)) # Nuestra ya conocida capa densa
#Ahora si, una capa densa con relu
model.add(keras.layers.Dense(128, activation=tf.nn.relu)) # Nuestra ya conocida capa densa

#Una última capa que tiene 1 elemento. La sigmoid se identifica más con una probabilidad.
model.add(keras.layers.Dense(num_clases, activation=tf.nn.softmax))

model.summary()

Model: "sequential_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_31 (Embedding)    (None, None, 64)          640000    
                                                                 
 dropout_15 (Dropout)        (None, None, 64)          0         
                                                                 
 conv1d_11 (Conv1D)          (None, None, 16)          3088      
                                                                 
 global_average_pooling1d_16  (None, 16)               0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_56 (Dense)            (None, 128)               2176      
                                                                 
 dense_57 (Dense)            (None, 128)               16512     
                                                     

In [ ]:
model.compile(optimizer='adam', 
             loss='binary_crossentropy', 
             metrics=['accuracy'])


history = model.fit(X_train,
                    y_train,
                    epochs=50, 
                    batch_size=512, 
                    verbose=1,
                    validation_split=0.1)

Epoch 1/50
16/16 [==============================] - 3s 51ms/step - loss: 0.1032 - accuracy: 0.3390 - val_loss: 0.1004 - val_accuracy: 0.3315
Epoch 2/50
16/16 [==============================] - 0s 19ms/step - loss: 0.0919 - accuracy: 0.3540 - val_loss: 0.0770 - val_accuracy: 0.3315
Epoch 3/50
16/16 [==============================] - 0s 10ms/step - loss: 0.0714 - accuracy: 0.3540 - val_loss: 0.0698 - val_accuracy: 0.3315
Epoch 4/50
16/16 [==============================] - 0s 10ms/step - loss: 0.0677 - accuracy: 0.3540 - val_loss: 0.0683 - val_accuracy: 0.3315
Epoch 5/50
16/16 [==============================] - 0s 11ms/step - loss: 0.0660 - accuracy: 0.3604 - val_loss: 0.0665 - val_accuracy: 0.3682
Epoch 6/50
16/16 [==============================] - 0s 9ms/step - loss: 0.0636 - accuracy: 0.3990 - val_loss: 0.0632 - val_accuracy: 0.4138
Epoch 7/50
16/16 [==============================] - 0s 9ms/step - loss: 0.0601 - accuracy: 0.4626 - val_loss: 0.0595 - val_accuracy: 0.4761
Epoch 8/50
16/1

In [ ]:
results = model.evaluate(X_test, y_test)
print(results)
acurracyExp1=results[1]
myTable.add_row(["CNN 1D", "{0:.5f}".format(acurracyExp1)])
print(myTable)

71/71 [==============================] - 0s 3ms/step - loss: 0.0401 - accuracy: 0.6790
[0.040084440261125565, 0.6789848804473877]
+-----------------+----------------+
|   Experimento   |  Test Accuracy |
+-----------------+----------------+
| Media Embedings |    0.70837     |
|       LSTM      |    0.68878     |
|      CNN 1D     |    0.67898     |
+-----------------+----------------+
